In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

In [ ]:
%pip install gdown

In [ ]:
RANDOM_STATE = 42

In [ ]:
CARS_FILE_ID = '1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI'

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

!gdown --id {CARS_FILE_ID}

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI
From (redirected): https://drive.google.com/uc?id=1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI&confirm=t&uuid=27256348-6c25-4b37-aa9b-68da2bc445db
To: /content/dataset.csv
100% 1.01G/1.01G [00:17<00:00, 57.7MB/s]


In [ ]:
df = pd.read_csv('dataset.csv')
print(df.shape)

(604047, 24)


In [ ]:
df_with_na_column = df[df['engine_displacement'].isna()]
df_with_na_column

,production_year,mileage,condition,owners_number,pts_original,horse_power,accidents_resolution,region,seller_type,brand,...,engine_displacement,engine_power,fuel_rate,steering_wheel,price,price_segment,tags,auto_class,equipment,complectation_available_options
210905,2003,175000,CONDITION_OK,0,True,NaN,NaN,Чебаркуль,PRIVATE,Hyundai,...,NaN,NaN,NaN,LEFT,380000,MEDIUM,available_for_checkup;pts_original;real_photo;...,NaN,seats-5,NaN


In [ ]:
df.drop(210905, inplace=True)

In [ ]:
df['fuel_rate'].fillna(df['fuel_rate'].median(), inplace=True)

<ipython-input-9-b50b155cf67a>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['fuel_rate'].fillna(df['fuel_rate'].median(), inplace=True)


In [ ]:
df['pts_original'].fillna(True, inplace=True)
df['accidents_resolution'].fillna('OK', inplace=True)
df['auto_class'].fillna('NOT SPECIFIED', inplace=True)

<ipython-input-10-f268849e3c66>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['pts_original'].fillna(True, inplace=True)
<ipython-input-10-f268849e3c66>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pts_original'].fillna(True, inplace=True)
<ipython-input-10-f268849e3c66>:2: FutureWarning: A value 

In [ ]:
df.drop('horse_power', axis=1, inplace=True)

In [ ]:
def get_unique_values(series, sep=';'):
    uniq = set()
    for cell in series.dropna():
        for piece in cell.split(sep):
            s = piece.strip()
            if s:
                uniq.add(s)
    return np.array(list(uniq))

all_tags = get_unique_values(df['tags'])
all_options = get_unique_values(df['complectation_available_options'])
all_equipments = get_unique_values(df['equipment'])

In [ ]:
def create_binary_features(df, column, unique_values, sep=';'):
    return (
        df[column]
        .str.get_dummies(sep=sep)
        .reindex(columns=unique_values, fill_value=0)
        .astype('int8')
    )

tags_dummies = create_binary_features(df, 'tags', all_tags)
options_dummies = create_binary_features(df, 'complectation_available_options', all_options)
equipment_dummies = create_binary_features(df, 'equipment', all_equipments)

In [ ]:
tags_dummies = tags_dummies.astype('Sparse[int]')
options_dummies = options_dummies.astype('Sparse[int]')
equipment_dummies = equipment_dummies.astype('Sparse[int]')

In [ ]:
full_df = pd.concat([
    df.drop(columns=['tags', 'complectation_available_options', 'equipment']),
    tags_dummies,
    options_dummies.drop(columns=['condition']),
    equipment_dummies.drop(columns=['condition'])
], axis=1)

In [ ]:
duplicated_cols = full_df.columns[full_df.columns.duplicated()].unique()
for col in duplicated_cols:
    full_df[col] = full_df[col].astype("int8")

In [ ]:
import numpy as np

names = full_df.columns[full_df.columns.duplicated()].unique()
new_cols = {}

for name in names:
    cols_i = [col for col in full_df.columns if col == name]
    sub_df = full_df.loc[:, cols_i].astype('int8')   # DataFrame shape (n_rows, k)

    # берём максимум по строкам → Series shape (n_rows,)
    max_series = sub_df.max(axis=1).astype('int8')

    new_cols[name] = max_series

    # удаляем *все* старые колонки с этим именем
    full_df.drop(columns=cols_i, inplace=True)

# теперь присваиваем новые колонки–Series (они одномерны!)
for name, s in new_cols.items():
    full_df[name] = s


<ipython-input-17-65bc9051fcec>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
<ipython-input-17-65bc9051fcec>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
<ipython-input-17-65bc9051fcec>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
<

In [ ]:
full_df.sample(10)

,production_year,mileage,condition,owners_number,accidents_resolution,region,seller_type,brand,model,body_type,...,steel-wheels,ptf,central-airbag,esp,body-kit,mirrors-heat,immo,lane-keeping-assist,drl,projection-display
436947,2024,0,CONDITION_OK,0,OK,Санкт-Петербург,COMMERCIAL,Skoda,Karoq,ALLROAD_5_DOORS,...,0,0,0,1,0,1,1,0,1,0
329729,2020,40460,CONDITION_OK,1,ERROR,Москва,PRIVATE,Mercedes-Benz,E-Класс,COUPE_HARDTOP,...,0,0,0,1,0,1,1,0,0,0
343497,2000,380000,CONDITION_OK,4,OK,Москва,PRIVATE,Mercedes-Benz,M-Класс,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
439784,2019,170000,CONDITION_OK,1,ERROR,Москва,PRIVATE,Skoda,Octavia,LIFTBACK,...,0,1,0,1,0,0,1,0,1,0
550845,2023,107,CONDITION_OK,1,OK,Краснодар,PRIVATE,Lada (ВАЗ),Largus,WAGON_5_DOORS,...,1,0,0,0,0,1,1,0,1,0
431808,2015,110727,CONDITION_OK,1,OK,Уфа,COMMERCIAL,Renault,Sandero,HATCHBACK_5_DOORS,...,1,1,0,0,0,1,1,0,0,0
167063,2024,0,CONDITION_OK,0,OK,Химки,COMMERCIAL,Genesis,GV80 Coupe,ALLROAD_5_DOORS,...,0,0,0,1,0,1,1,1,1,1
391523,2022,13000,CONDITION_OK,0,OK,Владивосток,COMMERCIAL,Nissan,X-Trail,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
210052,2024,100,CONDITION_OK,1,OK,Новосибирск,PRIVATE,Hyundai,Elantra,SEDAN,...,0,0,0,0,0,0,0,0,0,0
46816,2024,34,CONDITION_OK,1,OK,Москва,COMMERCIAL,BMW,X5,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

numeric_df = full_df.select_dtypes(include=[np.number])
numeric_df = numeric_df.loc[:, numeric_df.nunique() > 1]

corr = numeric_df.corr().abs().fillna(0)

dist = 1 - corr

dist = (dist + dist.T) / 2
np.fill_diagonal(dist.values, 0)

dist = dist.clip(lower=0)

dist_vect = squareform(dist)
Z = hierarchy.linkage(dist_vect, method='average')

max_d = 0.4
clusters = hierarchy.fcluster(Z, t=max_d, criterion='distance')

cluster_df = pd.DataFrame({'feature': numeric_df.columns, 'cluster': clusters})
to_keep = ['production_year', 'mileage', 'owners_number', 'doors_count', 'seats', 'engine_displacement', 'engine_power', 'fuel_rate']
for c in cluster_df['cluster'].unique():
    members = cluster_df.loc[cluster_df['cluster'] == c, 'feature']
    avg_corr = corr.loc[members, members].mean().sort_values(ascending=False)
    to_keep.append(avg_corr.index[0])
to_keep = list(dict.fromkeys(to_keep))

to_drop = [col for col in numeric_df.columns if col not in to_keep]
df_reduced = full_df.drop(columns=to_drop)

print(f"Удалено числовых коррелирующих признаков: {len(to_drop)}")
print(f"Осталось признаков в df_reduced: {df_reduced.shape[1]} из {full_df.shape[1]}")

Удалено числовых коррелирующих признаков: 112
Осталось признаков в df_reduced: 304 из 416


In [ ]:
df_reduced.sample(10)

,production_year,mileage,condition,owners_number,accidents_resolution,region,seller_type,brand,model,body_type,...,sport-pedals,tja,isofix-front,21-inch-wheels,leather-gear-stick,steel-wheels,ptf,central-airbag,body-kit,projection-display
518746,2007,33000,CONDITION_OK,1,ERROR,Аксай,PRIVATE,Lada (ВАЗ),2107,SEDAN,...,0,0,0,0,0,0,0,0,0,0
224119,2021,23211,CONDITION_OK,1,OK,Москва,PRIVATE,Hyundai,Solaris,SEDAN,...,0,0,0,0,0,0,1,0,0,0
351020,2021,6122,CONDITION_OK,1,OK,Уссурийск,COMMERCIAL,Mini,Countryman,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
424138,2016,4500,CONDITION_OK,1,ERROR,Погар,PRIVATE,Renault,Kaptur,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
89776,2011,245600,CONDITION_OK,3,OK,Тверь,PRIVATE,Chevrolet,Aveo,SEDAN,...,0,0,0,0,0,0,0,0,0,0
297634,2023,18300,CONDITION_OK,1,OK,Москва,PRIVATE,Lexus,RX,ALLROAD_5_DOORS,...,0,0,0,1,1,0,1,0,0,0
449720,2024,0,CONDITION_OK,0,OK,Москва,COMMERCIAL,Soueast,S07,ALLROAD_5_DOORS,...,0,1,0,0,0,0,1,0,0,0
49847,2020,121000,CONDITION_OK,1,OK,Москва,PRIVATE,BMW,X6,ALLROAD_5_DOORS,...,0,0,0,1,1,0,1,0,1,1
48872,2024,0,CONDITION_OK,0,OK,Краснодар,COMMERCIAL,BMW,X6,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
846,1999,363500,CONDITION_OK,3,OK,Уфа,PRIVATE,Alfa Romeo,166,SEDAN,...,0,0,0,0,0,0,1,0,0,0


### Добавим новые признаки

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
df_mod = df_reduced.copy()

In [ ]:
df_mod["age"] = 2025 - df_mod["production_year"]

In [ ]:
numeric_cols = ['mileage', 'engine_displacement', 'engine_power', 'age']

In [ ]:
df_mod['log_price'] = np.log1p(df['price'])

In [ ]:
df_mod.drop(columns=['price'], inplace=True)

Прологарифмируем некоторые признаки

In [ ]:
for col in numeric_cols:
    df_mod[f'log_{col}'] = np.log1p(df_mod[col])

Возьмем корни от числовых признаков

In [ ]:
for col in numeric_cols:
    df_mod[f'square_{col}'] = df_mod[col]**2

Полиномиальные преобразования

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_array = poly.fit_transform(df_mod[numeric_cols].fillna(0))
poly_feature_names = poly.get_feature_names_out(numeric_cols)
poly_df = pd.DataFrame(poly_array, columns=poly_feature_names, index=df_mod.index)

In [ ]:
poly_new = poly_df.drop(columns=[c for c in poly_df.columns if c in numeric_cols])
df_mod = pd.concat([df_mod, poly_new.add_prefix('poly_')], axis=1)

Базовые отношения двигателя

In [ ]:
df_mod['power_to_disp'] = df_mod['engine_power'] / df_mod['engine_displacement'].clip(lower=0.01)

Арифметичесике преобразования

In [ ]:
# Произведения признаков
df_mod['mileage_x_power'] = df_mod['mileage'] * df_mod['engine_power']
df_mod['disp_x_power'] = df_mod['engine_displacement'] * df_mod['engine_power']
df_mod['owners_x_power'] = df_mod['owners_number'] * df_mod['engine_power']
df_mod["age_x_power"]    = df_mod["age"] * df_mod["engine_power"]
df_mod["age_x_mileage"]  = df_mod["age"] * df_mod["mileage"]

# Частные признаков
df_mod['power_div_mileage'] = df_mod['engine_power'] / df_mod['mileage'].clip(lower=0.01)
df_mod['disp_per_mile'] = df_mod['engine_displacement'] / df_mod['mileage'].clip(lower=0.01)
df_mod['mileage_per_hp'] = df_mod['mileage'] / df_mod['engine_power'].clip(lower=0.01)
df_mod['milage_per_year'] = df_mod['mileage'] / df_mod['age'].clip(lower=0.01)
df_mod['power_div_owners'] = df_mod['engine_power'] / df_mod['owners_number'].clip(lower=0.01)
df_mod['disp_div_owners'] = df_mod['engine_displacement'] / df_mod['owners_number'].clip(lower=0.01)
df_mod['disp_div_fuel'] = df_mod['engine_displacement'] / df_mod['fuel_rate'].clip(lower=0.01)
df_mod['fuel_div_disp'] = df_mod['fuel_rate'] / df_mod['engine_displacement'].clip(lower=0.01)
df_mod['power_div_fuel'] = df_mod['engine_power'] / df_mod['fuel_rate'].clip(lower=0.01)
df_mod['fuel_div_power'] = df_mod['fuel_rate'] / df_mod['engine_power'].clip(lower=0.01)

# Лог-взаимодействие
df_mod['log_mileage_x_log_power'] = df_mod['log_mileage'] * df_mod['log_engine_power']
df_mod['log_age_x_log_power'] = df_mod['log_age'] * df_mod['log_engine_power']

# Разности признаков
df_mod['power_minus_disp'] = df_mod['engine_power'] - df_mod['engine_displacement']

In [ ]:
from pandas.api.types import is_numeric_dtype

# Отношение дверей к местам
df_mod['seats_num'] = df_mod['seats'].str.extract(r'(\d+)').astype(float)
df_mod['doors_to_seats_ratio'] = df_mod['doors_count'] / df_mod['seats_num'].clip(lower=0.01)

# Индикатор аварий
df_mod['has_accident'] = (
    ~df_mod['accidents_resolution'].fillna('none').str.lower().isin(['none', 'no', 'unknown'])
).astype(int)

binary_cols = [
    col for col in df.columns
    if is_numeric_dtype(df[col]) and
       set(df[col].dropna().unique()).issubset({0, 1})
]
df_mod['num_binary_features'] = df[binary_cols] \
    .astype(bool).sum(axis=1)

# Счетчики безопасности и комфорта
safety_keywords = ['airbag', 'detection', 'vsm', 'ldw', 'tja', 'rcta', 'ptf']
safety_cols = [c for c in df_mod.columns if any(k in c.lower() for k in safety_keywords)]
df_mod['safety_feature_count'] = df_mod[safety_cols] \
    .astype(bool).sum(axis=1)

comfort_keywords = ['heat', 'leather', 'massage', 'climate', 'vent', 'seat']
comfort_cols = [c for c in df_mod.columns if any(k in c.lower() for k in comfort_keywords)]
df_mod['comfort_feature_count'] = df_mod[comfort_cols] \
    .astype(bool).sum(axis=1)

# Количество preset-тегов
preset_cols = [c for c in df_mod.columns if c.startswith('preset')]
df_mod['num_presets'] = df_mod[preset_cols] \
    .astype(bool).sum(axis=1)

# Является ли машина новой
df_mod['is_new'] = ((df_mod['mileage'] < 1000) & (df_mod['owners_number'] <= 1)).astype(int)
df_mod['is_very_old'] = (df_mod['age'] > 20).astype(int)

df_mod['is_one_owner'] = (df_mod['owners_number'] == 1).astype(int)

Проверим, что нет дубликатов

In [ ]:
df_mod.columns.duplicated().sum()

np.int64(0)

Проверим, что не появилось пустых значений

In [ ]:
cols_with_na = df_mod.columns[df_mod.isna().any()].tolist()
print("Колонки с пропусками:", cols_with_na)

Колонки с пропусками: []


Сохраним датасет с новыми признаками

In [ ]:
df_mod.to_csv('dataset_with_new_features.csv', index=False)